# Data Analysis of Crimes in Toronto

This Notebook will go from taking the raw input of both the MCI Data and Homicide Data in the formats of CSV stored locally and merge these datasets. There is some checks run in the end to make sure the data is valid.

In [ ]:
Columns in MCI Data:
'X' - Not Required same as Long,
'Y' - Not Required same as Lat,
'Index_' - Not Required (maybe incase we need to pick them in some order),
'event_unique_id' - Not Required,
'occurrencedate' - Required,
'reporteddate' - Not Required,
'premisetype' - Not Really,
'ucr_code' - Not Required,
'ucr_ext' - Not Required,
'offence' - Keeping this,
'reportedyear' - Not Required,
'reportedmonth'- Not Required,
'reportedday' - Not Required,
'reporteddayofyear' - Not Required,
'reporteddayofweek' - Not Required,
'reportedhour' - Not Required,
'occurrenceyear' - Not Required, Can be extracted from Occurrencedate,
'occurrencemonth' - Not Required, Same as above,
'occurrenceday' - Not Required, Same as above,
'occurrencedayofyear' - Not Required,
'occurrencedayofweek'- Not Required,
'occurrencehour' - Not Required,
'MCI' - Keeping this,
'Division' - Not Required,
'Hood_ID' - Not Required,
'Neighbourhood' - Required,
'Lat' - YES,
'Long' - YES,
'ObjectId' - Not Required

In [ ]:
Columns in Homicide Data:
'X',
'Y',
'Index_',
'Event_Unique_ID',
'Occurrence_year',
'Division',
'Homicide_Type',
'Occurrence_Date',
'Hood_ID',
'Neighbourhood',
'Lat',
'Long',
'ObjectId'

In [16]:
!pip3 install pandas

     |████████████████████████████████| 10.1MB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 15.0MB 9.3MB/s eta 0:00:011
     |████████████████████████████████| 512kB 16.6MB/s eta 0:00:01


In [6]:
!pip3 install requests

  Using cached https://files.pythonhosted.org/packages/51/bd/23c926cd341ea6b7dd0b2a00aba99ae0f828be89d72b2190f27c11d4b7fb/requests-2.22.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/14/2c/cd551d81dbe15200be1cf41cd03869a46fe7226e7450af7a6545bfc474c9/idna-2.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e6/60/247f23a7121ae632d62811ba7f273d0e58972d75e58a94d329d51550a47d/urllib3-1.25.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/18/b0/8146a4f8dd402f60744fa380bc73ca47303cccf8b9190fd16a827281eac2/certifi-2019.9.11-py2.py3-none-any.whl


In [64]:
!pip3 install pyyaml

     |████████████████████████████████| 266kB 2.4MB/s eta 0:00:01
  Stored in directory: /Users/satish127/Library/Caches/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built pyyaml


In [66]:
import requests
import pandas as pd
import ast
import yaml

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Using API to import Crime Data

The API returns the whole dataset and performing operations on this dataset gets the 

In [59]:
response = requests.get("https://opendata.arcgis.com/datasets/98f7dde610b54b9081dfca80be453ac9_0.geojson")

In [67]:
res_json = yaml.load(response.text)
print(len(res_json))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """Entry point for launching an IPython kernel.


KeyboardInterrupt: 

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Loading Crime Data Offline

In [4]:
main_df = pd.read_csv('MCI.csv')
homi_df = pd.read_csv('Homicide.csv')
print("Main Shape: ", main_df.shape)
print("Homi Shape: ", homi_df.shape)

Main Shape:  (167525, 29)
Homi Shape:  (1015, 13)


#### Columns to be dropped

In [8]:
drp = ['X','Y','Index_','event_unique_id','reporteddate',
 'premisetype','ucr_code','ucr_ext','reportedyear','occurrencedayofyear',
 'occurrencedayofweek','reportedmonth','reportedday','reporteddayofyear',
 'reporteddayofweek','reportedhour','occurrenceyear','occurrencemonth',
 'occurrenceday','occurrencehour','Division','Hood_ID','ObjectId']

main_df = main_df.drop(drp, axis=1)
for i in drp:
    if i in homi_df:
        homi_df = homi_df.drop(columns=[i])
print("Main Shape: ", main_df.shape)
print("Homi Shape: ", homi_df.shape)

Main Shape:  (167525, 8)
Homi Shape:  (1015, 7)


#### Merging the datasets

Updating the shape of Dataframes to merge

In [20]:
homi_df['MCI'] = 'Homicide'
homi_df['Offence'] = 'Homicide - ' + homi_df['Homicide_Type']
homi_df = homi_df.drop(columns=['Homicide_Type', 'Event_Unique_ID', 'Occurrence_year'])

In [32]:
main_df['Offence'] = main_df['offence']
main_df['Occurrence_Date'] = main_df['occurrencedate']
main_df = main_df.drop(['occurrencedate', 'offence'], axis =1)

In [37]:
dataset = pd.concat([main_df, homi_df], axis=0, sort=True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


#### Saving the Dataset outside, Could be directly used as well

In [42]:
dataset.to_csv('Crime_Data.csv', index=False)

#### Random Check and Analysis of the data

In [43]:
list(dataset)

['Lat', 'Long', 'MCI', 'Neighbourhood', 'Occurrence_Date', 'Offence']

In [52]:
for i in list(dataset):
    print(len(list(dataset[i])))
    if i == 'Offence':
        print(set(dataset[i]))
    elif i == 'MCI':
        print(set(dataset[i]))
    elif i == 'Neighbourhood':
        print(set(dataset[i]))

168540
168540
168540
168540
168540
168540
